In [ ]:
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString
import numpy as np
import pandas as pd

In [14]:
#dummydataframe für test

import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

# Beispielhafte Daten
daten = [
    {
        "Abschnittsname": "Station A – Station B",
        "Von": "Station A",
        "Nach": "Station B",
        "geometry": LineString([
            (2650000, 1140000, 800),
            (2650100, 1140100, 820),
            (2650200, 1140150, 850),
            (2650300, 1140200, 860),
            (2650400, 1140300, 870),
            (2650500, 1140400, 880),
        ])
    },
    {
        "Abschnittsname": "Station B – Station C",
        "Von": "Station B",
        "Nach": "Station C",
        "geometry": LineString([
            (2650500, 1140400, 880),
            (2650600, 1140350, 860),
            (2650700, 1140300, 840),
            (2650800, 1140250, 820),
            (2650900, 1140200, 800),
            (2651000, 1140100, 780),
        ])
    },
]

# Erstellen des GeoDataFrames
gdf = gpd.GeoDataFrame(daten, crs="EPSG:2056")

gdf


,Abschnittsname,Von,Nach,geometry
0,Station A – Station B,Station A,Station B,"LINESTRING Z (2650000 1140000 800, 2650100 114..."
1,Station B – Station C,Station B,Station C,"LINESTRING Z (2650500 1140400 880, 2650600 114..."


In [16]:
#berechnet die Länge der Geometrie in Kilometer aus dem Linestring
gdf["Laenge [km]"] = gdf.length/1000
gdf

,Abschnittsname,Von,Nach,geometry,Laenge [km]
0,Station A – Station B,Station A,Station B,"LINESTRING Z (2650000 1140000 800, 2650100 114...",0.647871
1,Station B – Station C,Station B,Station C,"LINESTRING Z (2650500 1140400 880, 2650600 114...",0.588635


In [ ]:
# Example GeoDataFrame with one 3D LineString
gdf = gpd.GeoDataFrame({
    'geometry': [wkt.loads("LINESTRING Z (0 0 100, 1000 0 110, 2000 0 130)")]
}, geometry='geometry')

# Create empty columns
gdf['km'] = 0.0
gdf['hm'] = 0.0
gdf['Leistungskm'] = 0.0

# Loop through each row
for idx, row in gdf.iterrows():
    line = row.geometry
    
    coords = np.array(line.coords)  # extract coordinate array
    
    # Get XY differences (horizontal segments)
    delta_xy = np.diff(coords[:, :2], axis=0)
    d_segs = np.linalg.norm(delta_xy, axis=1)  # in meters
    
    # Get Z differences (vertical height changes)
    delta_z = np.diff(coords[:, 2])
    h_segs = delta_z.copy()
    h_segs[h_segs < 0] = 0  # only positive elevation gain
    
    # Total horizontal distance (in km)
    km = np.sum(d_segs) / 1000.0
    
    # Total height gain (in m)
    hm = np.sum(h_segs)
    
    # Leistungskilometer per segment and total
    leistungskm_segs = d_segs / 1000.0 + h_segs / 100.0
    leistungskm = np.sum(leistungskm_segs)
    
    # Assign to DataFrame
    gdf.at[idx, 'km'] = km
    gdf.at[idx, 'hm'] = hm
    gdf.at[idx, 'Leistungskm'] = leistungskm

gdf[['km', 'hm', 'Leistungskm']]
